In [1]:
from pyspark import SparkContext #Library
sc = SparkContext("local[*]", "temp") #Start a spart context


In [2]:
posts_lines = sc.textFile("file:///home/vagrant/miniprojects/spark/part/allPosts/*.gz") #create an RDD from file
print posts_lines
posts_lines.count()


MapPartitionsRDD[1] at textFile at NativeMethodAccessorImpl.java:-2


212990

In [3]:
#Import parsing library.
import xml.etree.ElementTree as ET
import numpy as np
from datetime import datetime


In [4]:
def parseQuestion(line):
    if '<row' in line:
        try:
            root = ET.fromstring(line.encode('utf8'))
        except:
            pass
            return ("Empty")
            
        if root != '':
            if "CreationDate" in root.attrib:
                try:
                    date = root.attrib["CreationDate"]
                except:
                    pass
                    return ("Empty")
                if "AcceptedAnswerId" in root.attrib:
                    return(root.attrib["AcceptedAnswerId"], date)
                else:
                    return("Empty")
            else:
                return("Empty")
        else:
            return("Empty")
    else:
        return("Empty")
            
        

In [5]:
questions = posts_lines.map(parseQuestion).filter(lambda x: x!= 'Empty')
print questions
print questions.take(40)
questions.count()


PythonRDD[3] at RDD at PythonRDD.scala:43
[('105339', '2014-06-04T13:31:24.160'), ('101146', '2014-06-04T14:28:42.040'), ('101185', '2014-06-04T15:00:58.427'), ('101194', '2014-06-04T15:14:33.713'), ('101175', '2014-06-04T16:02:39.143'), ('101163', '2014-06-04T16:15:59.327'), ('101191', '2014-06-04T18:52:46.957'), ('101323', '2014-06-05T01:48:33.763'), ('104912', '2014-06-05T02:53:10.147'), ('101227', '2014-06-05T03:08:06.267'), ('101295', '2014-06-05T04:44:59.133'), ('104597', '2014-06-05T05:53:16.250'), ('101256', '2014-06-05T07:12:05.407'), ('101260', '2014-06-05T09:07:11.243'), ('102644', '2014-06-05T09:25:35.993'), ('101270', '2014-06-05T10:23:00.450'), ('101290', '2014-06-05T10:44:37.823'), ('106333', '2014-06-05T12:33:49.403'), ('101319', '2014-06-05T15:30:57.313'), ('101336', '2014-06-05T16:59:21.787'), ('101519', '2014-06-05T20:36:21.673'), ('101348', '2014-06-05T20:39:36.447'), ('101366', '2014-06-05T21:45:22.947'), ('101379', '2014-06-06T00:46:45.240'), ('115931', '2014-06-0

17304

In [6]:
def parseAnswer(line):
    if '<row' in line:
        try:
            root = ET.fromstring(line.encode('utf8'))
        except:
            pass
            return ("Empty")
            
        if root != '':
            if "CreationDate" in root.attrib:
                try:
                    date = root.attrib["CreationDate"]
                except:
                    pass
                    return ("Empty")
                if "AcceptedAnswerId" not in root.attrib:
                    return(root.attrib["Id"], date)
                else:
                    return("Empty")
            else:
                return("Empty")
        else:
            return("Empty")
    else:
        return("Empty")
            
        

In [7]:
answers = posts_lines.map(parseAnswer).filter(lambda x: x!= 'Empty')
print answers
print answers.take(40)
answers.count()


PythonRDD[6] at RDD at PythonRDD.scala:43
[('101120', '2014-06-04T13:18:18.120'), ('101123', '2014-06-04T13:43:15.913'), ('101124', '2014-06-04T13:57:07.657'), ('101126', '2014-06-04T14:14:48.753'), ('101127', '2014-06-04T14:19:20.430'), ('101128', '2014-06-04T14:25:45.873'), ('101130', '2014-06-04T14:30:15.493'), ('101131', '2014-06-04T14:35:08.560'), ('101132', '2014-06-04T14:35:51.100'), ('101133', '2014-06-04T14:41:11.140'), ('101134', '2014-06-04T14:48:17.310'), ('101138', '2014-06-04T15:07:20.207'), ('101139', '2014-06-04T15:07:48.407'), ('101140', '2014-06-04T15:07:55.817'), ('101143', '2014-06-04T15:13:20.170'), ('101145', '2014-06-04T15:24:10.153'), ('101146', '2014-06-04T15:24:18.320'), ('101147', '2014-06-04T15:31:46.327'), ('101148', '2014-06-04T15:33:09.367'), ('101149', '2014-06-04T15:34:04.377'), ('101150', '2014-06-04T15:41:41.950'), ('101151', '2014-06-04T15:46:59.423'), ('101152', '2014-06-04T15:52:04.677'), ('101155', '2014-06-04T16:09:49.117'), ('101156', '2014-06-0

91437

In [19]:
from datetime import datetime
import time

def dateConversion (date1, date2):  
    date1_obj = dat etime.strptime(date1, "%Y-%m-%dT%H:%M:%S.%f")
    date2_obj = datetime.strptime(date2, "%Y-%m-%dT%H:%M:%S.%f")
    diff = date2_obj - date1_obj
    diff_sec = diff.total_seconds()
    hour = date1_obj.hour
    if diff_sec < 10800:
        return(hour, (1.,1.))
    else:
        return(hour, (0.,1.))
  
    

In [20]:
joint = questions.join(answers).map(lambda x: x[1]).map(lambda x: dateConversion(x[0],x[1])) \
         .reduceByKey(lambda x,y: np.add(x,y)).map(lambda x: (x[0],x[1][0]/x[1][1]))\
        .sortByKey().map(lambda x: x[1]).collect()
print joint[:25]

[0.45046728971962618, 0.44814814814814813, 0.3605577689243028, 0.37991266375545851, 0.40284360189573459, 0.41249999999999998, 0.45974025974025973, 0.4673684210526316, 0.46168224299065419, 0.49528301886792453, 0.51575931232091687, 0.54456824512534818, 0.53473132372214938, 0.53107960741548532, 0.52380952380952384, 0.53680078508341511, 0.54757281553398063, 0.47995991983967934, 0.52021857923497272, 0.5462012320328542, 0.51964085297418627, 0.51567944250871078, 0.46153846153846156, 0.47004608294930877]


In [ ]:
check = questions.join(answers).map(lambda x: x[1]).map(lambda x: dateConversion(x[0],x[1])) \
         .reduceByKey(lambda x,y: np.add(x,y)).collect()
print check[:2]

In [61]:
from datetime import datetime
import time
date1 = datetime.strptime('2013-06-10T13:43:24.690', "%Y-%m-%dT%H:%M:%S.%f")
date2 = datetime.strptime('2013-06-12T05:26:54.733', "%Y-%m-%dT%H:%M:%S.%f")
print date1, date2
diff = date2-date1
print type(diff)
print diff.total_seconds()
print date1.hour

   
    

2013-06-10 13:43:24.690000 2013-06-12 05:26:54.733000
<type 'datetime.timedelta'>
143010.043
13
